In [49]:
library(tidyverse)
library(RPostgreSQL)
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [2]:
drv <- dbDriver("PostgreSQL")

In [3]:
user <- pw <- "datathon"
dbname <- "mimic"

In [4]:
con <- dbConnect(drv, dbname = dbname, user = user, password = pw)

In [41]:
que <- "
    SELECT * FROM (SELECT subject_id, hadm_id, icustay_id, intime, outtime,
        MIN(intime) over (PARTITION BY subject_id) AS first
        FROM icustays) AS icu
    WHERE intime = first
    LIMIT 10;
"
quedf <- dbGetQuery(con, que)
quedf

subject_id,hadm_id,icustay_id,intime,outtime,first
2,163353,243653,2138-07-17 21:20:07,2138-07-17 23:32:21,2138-07-17 21:20:07
3,145834,211552,2101-10-20 19:10:11,2101-10-26 20:43:09,2101-10-20 19:10:11
4,185777,294638,2191-03-16 00:29:31,2191-03-17 16:46:31,2191-03-16 00:29:31
5,178980,214757,2103-02-02 06:04:24,2103-02-02 08:06:00,2103-02-02 06:04:24
6,107064,228232,2175-05-30 21:30:54,2175-06-03 13:39:54,2175-05-30 21:30:54
7,118037,278444,2121-05-23 15:35:29,2121-05-23 22:01:00,2121-05-23 15:35:29
8,159514,262299,2117-11-20 12:36:10,2117-11-21 14:24:55,2117-11-20 12:36:10
9,150750,220597,2149-11-09 13:07:02,2149-11-14 20:52:14,2149-11-09 13:07:02
10,184167,288409,2103-06-28 11:39:05,2103-07-06 13:51:43,2103-06-28 11:39:05
11,194540,229441,2178-04-16 06:19:32,2178-04-17 20:21:05,2178-04-16 06:19:32


In [44]:
first_icu_que <- "
    SELECT icu.subject_id, icu.hadm_id, icu.icustay_id,
        icu.intime, icu.outtime,
        adm.admittime, adm.dischtime, adm.deathtime
    FROM
        (SELECT * FROM (SELECT subject_id, hadm_id, icustay_id, intime, outtime,
        MIN(intime) over (PARTITION BY subject_id) AS first
        FROM icustays) AS icu
        WHERE intime = first) AS icu,
        admissions AS adm
    WHERE adm.subject_id = icu.subject_id AND
        adm.hadm_id = icu.hadm_id;
"

In [45]:
first_icu <- dbGetQuery(con, first_icu_que)
head(first_icu)

subject_id,hadm_id,icustay_id,intime,outtime,admittime,dischtime,deathtime
2,163353,243653,2138-07-17 21:20:07,2138-07-17 23:32:21,2138-07-17 19:04:00,2138-07-21 15:48:00,NA
3,145834,211552,2101-10-20 19:10:11,2101-10-26 20:43:09,2101-10-20 19:08:00,2101-10-31 13:58:00,NA
4,185777,294638,2191-03-16 00:29:31,2191-03-17 16:46:31,2191-03-16 00:28:00,2191-03-23 18:41:00,NA
5,178980,214757,2103-02-02 06:04:24,2103-02-02 08:06:00,2103-02-02 04:31:00,2103-02-04 12:15:00,NA
6,107064,228232,2175-05-30 21:30:54,2175-06-03 13:39:54,2175-05-30 07:15:00,2175-06-15 16:00:00,NA
7,118037,278444,2121-05-23 15:35:29,2121-05-23 22:01:00,2121-05-23 15:05:00,2121-05-27 11:57:00,NA


In [51]:
fst_icu <- first_icu %>%
    mutate(intime = ymd_hms(intime),
           outtime = ymd_hms(outtime),
           admittime = ymd_hms(admittime),
           dischtime = ymd_hms(dischtime),
           deathtime = ymd_hms(deathtime))
head(fst_icu)

subject_id,hadm_id,icustay_id,intime,outtime,admittime,dischtime,deathtime
2,163353,243653,2138-07-17 21:20:07,2138-07-17 23:32:21,2138-07-17 19:04:00,2138-07-21 15:48:00,NA
3,145834,211552,2101-10-20 19:10:11,2101-10-26 20:43:09,2101-10-20 19:08:00,2101-10-31 13:58:00,NA
4,185777,294638,2191-03-16 00:29:31,2191-03-17 16:46:31,2191-03-16 00:28:00,2191-03-23 18:41:00,NA
5,178980,214757,2103-02-02 06:04:24,2103-02-02 08:06:00,2103-02-02 04:31:00,2103-02-04 12:15:00,NA
6,107064,228232,2175-05-30 21:30:54,2175-06-03 13:39:54,2175-05-30 07:15:00,2175-06-15 16:00:00,NA
7,118037,278444,2121-05-23 15:35:29,2121-05-23 22:01:00,2121-05-23 15:05:00,2121-05-27 11:57:00,NA


In [53]:
mort_icu <- fst_icu %>%
    mutate(onemth = intime + months(1),
           thrmth = intime + months(3),
           sixmth = intime + months(6))
head(mort_icu)

subject_id,hadm_id,icustay_id,intime,outtime,admittime,dischtime,deathtime,onemth,thrmth,sixmth
2,163353,243653,2138-07-17 21:20:07,2138-07-17 23:32:21,2138-07-17 19:04:00,2138-07-21 15:48:00,NA,2138-08-17 21:20:07,2138-10-17 21:20:07,2139-01-17 21:20:07
3,145834,211552,2101-10-20 19:10:11,2101-10-26 20:43:09,2101-10-20 19:08:00,2101-10-31 13:58:00,NA,2101-11-20 19:10:11,2102-01-20 19:10:11,2102-04-20 19:10:11
4,185777,294638,2191-03-16 00:29:31,2191-03-17 16:46:31,2191-03-16 00:28:00,2191-03-23 18:41:00,NA,2191-04-16 00:29:31,2191-06-16 00:29:31,2191-09-16 00:29:31
5,178980,214757,2103-02-02 06:04:24,2103-02-02 08:06:00,2103-02-02 04:31:00,2103-02-04 12:15:00,NA,2103-03-02 06:04:24,2103-05-02 06:04:24,2103-08-02 06:04:24
6,107064,228232,2175-05-30 21:30:54,2175-06-03 13:39:54,2175-05-30 07:15:00,2175-06-15 16:00:00,NA,2175-06-30 21:30:54,2175-08-30 21:30:54,2175-11-30 21:30:54
7,118037,278444,2121-05-23 15:35:29,2121-05-23 22:01:00,2121-05-23 15:05:00,2121-05-27 11:57:00,NA,2121-06-23 15:35:29,2121-08-23 15:35:29,2121-11-23 15:35:29


In [71]:
mort_fst_icu <- mort_icu %>%
    mutate(icu_mort = (!is.na(deathtime) & deathtime >= outtime & deathtime <= outtime)) %>%
    mutate(h_mort = (!is.na(deathtime) & (deathtime >= admittime & deathtime <= dischtime | icu_mort))) %>%
    mutate(one_mort = (!is.na(deathtime) & deathtime <= onemth)) %>%
    mutate(thr_mort = (!is.na(deathtime) & deathtime <= thrmth)) %>%
    mutate(six_mort = (!is.na(deathtime) & deathtime <= sixmth))
head(mort_fst_icu)

subject_id,hadm_id,icustay_id,intime,outtime,admittime,dischtime,deathtime,onemth,thrmth,sixmth,icu_mort,h_mort,one_mort,thr_mort,six_mort
2,163353,243653,2138-07-17 21:20:07,2138-07-17 23:32:21,2138-07-17 19:04:00,2138-07-21 15:48:00,NA,2138-08-17 21:20:07,2138-10-17 21:20:07,2139-01-17 21:20:07,FALSE,FALSE,FALSE,FALSE,FALSE
3,145834,211552,2101-10-20 19:10:11,2101-10-26 20:43:09,2101-10-20 19:08:00,2101-10-31 13:58:00,NA,2101-11-20 19:10:11,2102-01-20 19:10:11,2102-04-20 19:10:11,FALSE,FALSE,FALSE,FALSE,FALSE
4,185777,294638,2191-03-16 00:29:31,2191-03-17 16:46:31,2191-03-16 00:28:00,2191-03-23 18:41:00,NA,2191-04-16 00:29:31,2191-06-16 00:29:31,2191-09-16 00:29:31,FALSE,FALSE,FALSE,FALSE,FALSE
5,178980,214757,2103-02-02 06:04:24,2103-02-02 08:06:00,2103-02-02 04:31:00,2103-02-04 12:15:00,NA,2103-03-02 06:04:24,2103-05-02 06:04:24,2103-08-02 06:04:24,FALSE,FALSE,FALSE,FALSE,FALSE
6,107064,228232,2175-05-30 21:30:54,2175-06-03 13:39:54,2175-05-30 07:15:00,2175-06-15 16:00:00,NA,2175-06-30 21:30:54,2175-08-30 21:30:54,2175-11-30 21:30:54,FALSE,FALSE,FALSE,FALSE,FALSE
7,118037,278444,2121-05-23 15:35:29,2121-05-23 22:01:00,2121-05-23 15:05:00,2121-05-27 11:57:00,NA,2121-06-23 15:35:29,2121-08-23 15:35:29,2121-11-23 15:35:29,FALSE,FALSE,FALSE,FALSE,FALSE


In [81]:
mort_sum <- mort_fst_icu %>%
    select_(.dots = c("subject_id", "hadm_id", "icustay_id", grep("mort", names(.), value = TRUE))) %>%
    mutate_(.dots = grep("mort", names(.), value = TRUE) %>%
                setNames(., .) %>%
                map(~ sprintf("~ as.integer(%s)", .x)) %>%
                map(~ as.formula(.x)))
head(mort_sum)

subject_id,hadm_id,icustay_id,icu_mort,h_mort,one_mort,thr_mort,six_mort
2,163353,243653,0,0,0,0,0
3,145834,211552,0,0,0,0,0
4,185777,294638,0,0,0,0,0
5,178980,214757,0,0,0,0,0
6,107064,228232,0,0,0,0,0
7,118037,278444,0,0,0,0,0


In [82]:
data.table::fwrite(mort_sum, "/Data/common/mort_sum.csv")

In [83]:
mort_sum %>%
    summarise(i = sum(icu_mort, na.rm = TRUE) / n(),
              h = sum(h_mort, na.rm = TRUE) / n(),
              o = sum(one_mort, na.rm = TRUE) / n(),
              t = sum(thr_mort, na.rm = TRUE) / n(),
              s = sum(six_mort, na.rm = TRUE) / n())

i,h,o,t,s
0.004776659,0.09379034,0.09021861,0.09443584,0.09422067
